In [ ]:
import tensorflow as tf

DDPG是google DeepMind团队提出的一种用于输出确定性动作的算法，它解决了Actor-Critic 神经网络每次参数更新前后都存在相关性，导致神经网络只能片面的看待问题这一缺点。同时也解决了DQN不能用于连续性动作的缺点。

## Deep Deterministic Policy Gradient

### 简介

DDPG即深度确定性策略梯度算法。也是一种可以解决连续性控制问题的方法。属于model-free、off-policy、policy-based的方法。

DDPG我们可以拆开来看，Deep，是说明需要**神经网络**。Deterministic的意思就是最终确定地**只输出一个动作**。Policy Gradient我们已经知道是**策略梯度算法**。DDPG可以看成是DQN的扩展版，不同的是，以往的DQN在最终输出的是一个动作向量，对于DDPG是最终确定地只输出一个动作。而且，DDPG让 DQN 可以扩展到连续的动作空间。

### 网络结构

DDPG的结构形式类似Actor-Critic。DDPG可以分为**策略网络**和**价值网络**两个大网络。DDPG延续DQN了固定目标网络的思想，每个网络再细分为目标网络和现实网络。不过目标网络的更新上有些不同。下面详细分析。

先来看策略网络，也就是Actor。Actor输出的是一个确定性的动作，产生这个确定性动作的网络定义为 @todo 。以往的policy gradient采取的是随机策略，每一次获取动作都需要对当前的最优策略的分布进行采样，而DDPG采取的则是确定性策略，直接通过函数 @todo 确定。Actor的估计网络就是 @todo ，@todo 就是神经网络的参数，这个估计网络就是用来输出实时的动作。此外，Actor还有一个相同结构但不同参数的目标网络，是用来更新价值网络Critic的。两个网络都是输出动作action。

再来看价值网络，也就是Critic。它的作用就是来拟合价值函数 @todo 。同样也有一个估计网络和一个目标网络。这两个网络在输出端都输出当前状态的价值 q-value，在输入端则有所不同。Critic的目标网络输入有两个参数，分别是当前状态的观测值和Actor的目标网络输出的动作action。Critic的估计网络的输入则是当前Actor的估计网络输出的动作action。目标网络是用来计算 @todo ,

下面我们通过一张图来直观的理解下整个过程：


我们可以看到，价值网络的更新是基于TD-error的梯度下降，Critic作为评委，一开始也不知道Actor输出的动作是否足够好，它也需要一步一步学习给出准确的打分，所以借助于目标网络拟合的下一时刻的价值 
 ，以及真实的收益r,我们可以得到 
 ，让 
 减去当前 
 求均方差，则可以构造出Loss。更新方法其实和DQN差不多的，唯一不同的地方在于目标网络的参数在DDPG算法中是缓慢更新的，而不是在DQN中每隔N步将现有网络的参数直接复制过来。

而策略网络（Actor）的更新是基于梯度上升，因为Actor的目标就是找到一个动作a能使得输出的价值Q最大，所以优化策略网络的梯度就是要最大化价值网络输出的这个Q值。Loss函数就加上一个负号方便最小化误差。

值得注意的是，DDQG也借鉴了DQN的经验回放 (Experience Replay)的技巧。DDPG同样会把一段时间的序列 
 存储到经验池。每次训练的时候，从经验池随机采样一个minibatch来训练就好。

算法流程
伪代码如下：


首先，初始化Actor和Critic以及其各自的目标网络共4个网络以及经验池replay buffer R。

在Actor网络输出动作时，DDPG通过添加随机噪声的方式实现exploration，可以让智能体更好的探索潜在的最优策略。之后是采取经验回放的技巧。把智能体与环境交互的数据 
 存储到R。随后每次训练从中随机采样一个minibatch。

在参数更新上，先利用Critic的目标网络 
 来计算目标值 
 ，利用 
 与当前Q值的均方误差构造损失函数，进行梯度更新。对于Actor的策略网络，其实就是把Actor的确定性动作函数代进Q-function的a，然后求梯度，最后是更新目标网络。

小结

简单来说：DQN + Actor-Critic => Deep Deterministic Policy Gradient (DDPG)。实际上DDPG其实更接近DQN，只是采用了类似Actor-Critic的结构。DDPG吸收了Actor-Critic中策略梯度单步更新的优点，同时还吸收了DQN对Q值估计的技巧。DDPG 最大的优势就是能够在**连续动作**上更有效地学习。

参考文献：

Silver, David, et al. "Deterministic policy gradient algorithms." ICML. 2014.（前序工作）

Lillicrap, Timothy P., et al. "Continuous control with deep reinforcement learning." arXiv preprint arXiv:1509.02971 (2015)